In [1]:
from config import hf_cache_dir
import transformers
import torch
import os
from transformers import pipeline

In [2]:
hf_cache_dir

'/workspace/data/huggingface-cache/hub'

In [3]:
pipe = pipeline(
    "text-generation", model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B",
    model_kwargs={"torch_dtype": torch.bfloat16, "cache_dir": hf_cache_dir},
    device_map="auto",
)

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

model-00002-of-000017.safetensors:   0%|          | 0.00/8.69G [00:00<?, ?B/s]

model-00001-of-000017.safetensors:   0%|          | 0.00/8.95G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [46]:
prompt = "What is the most populous city in the state where the author of The Call of the Wild was born in?"
#prompt = "how can identity protection services help protect me against identity theft"

messages = [
    {"role": "user", "content": prompt},
]

outputs = pipe(
    messages,
    max_new_tokens=2048,
    do_sample=False,
    temperature=0.6,
    top_p=0.9,
)
output = outputs[0]["generated_text"][-1]['content']
print(output)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Okay, so I need to figure out the most populous city in the state where the author of "The Call of the Wild" was born. Let me break this down step by step.

First, I should identify who the author of "The Call of the Wild" is. I remember that Jack London wrote that book. Yeah, that's right. So, Jack London is the author.

Next, I need to find out where Jack London was born. I think he was an American author, so he was born in the United States. I'm not exactly sure of the state, but I believe it's California. Let me confirm that. Yes, Jack London was born in San Francisco, California, in 1876.

Now, the question is asking for the most populous city in the state where he was born. Since he was born in California, I need to determine the most populous city in California. I know that California is a large state with many big cities, but the most populous one is Los Angeles. I've heard that Los Angeles is not only the most populous city in California but also in the United States, but I th

In [48]:
outputs[0]['generated_text'][-1]['content']

'Okay, so I need to figure out the most populous city in the state where the author of "The Call of the Wild" was born. Let me break this down step by step.\n\nFirst, I should identify who the author of "The Call of the Wild" is. I remember that Jack London wrote that book. Yeah, that\'s right. So, Jack London is the author.\n\nNext, I need to find out where Jack London was born. I think he was an American author, so he was born in the United States. I\'m not exactly sure of the state, but I believe it\'s California. Let me confirm that. Yes, Jack London was born in San Francisco, California, in 1876.\n\nNow, the question is asking for the most populous city in the state where he was born. Since he was born in California, I need to determine the most populous city in California. I know that California is a large state with many big cities, but the most populous one is Los Angeles. I\'ve heard that Los Angeles is not only the most populous city in California but also in the United State

# Using auto classes

In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-70B")

In [19]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-70B", cache_dir=hf_cache_dir)

Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

In [23]:
messages = [
    {"role": "user", "content": prompt},
]

# Apply chat template (converts messages to the model's expected format)
formatted_prompt = tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=True
)

# Tokenize the input
inputs = tokenizer(formatted_prompt, return_tensors="pt")

# Move inputs to the same device as model (if needed)
inputs = {k: v.to(model.device) for k, v in inputs.items()}

# Generate text
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=2048,
        do_sample=False,  # Set to True for sampling
        temperature=0.6,  # Only used when do_sample=True
        top_p=0.9,        # Only used when do_sample=True
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

# Decode the generated text
# Remove the input tokens to get only the generated part
generated_tokens = outputs[0][inputs['input_ids'].shape[1]:]
generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print("Generated text:")
print(generated_text)

# For debugging the <think> tag issue, decode without skipping special tokens
generated_text_with_special = tokenizer.decode(generated_tokens, skip_special_tokens=False)
print("\nGenerated text (with special tokens):")
print(generated_text_with_special)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generated text:
Okay, so I need to figure out the most populous city in the state where the author of "The Call of the Wild" was born. Let me break this down step by step.

First, I should identify who the author of "The Call of the Wild" is. I remember that Jack London wrote that book. Yeah, that's right. So, Jack London is the author.

Next, I need to find out where Jack London was born. I think he was an American author, so he was born in the United States. I'm not exactly sure of the state, but I believe it's California. Wait, wasn't he born in San Francisco? I think that's correct. So, his birthplace is San Francisco, California.

Now, the question is asking for the most populous city in the state where he was born. Since he was born in California, I need to determine the most populous city in California. I know that Los Angeles is a very big city in California, but I'm not entirely sure if it's the most populous. I think it is, but I should double-check. San Francisco is also a m

In [30]:
tokenizer.decode(inputs['input_ids'][0])

'<｜begin▁of▁sentence｜><｜begin▁of▁sentence｜><｜User｜>What is the most populous city in the state where the author of The Call of the Wild was born in?<｜Assistant｜><think>\n'

In [35]:
new_msg = [
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": '<think>' + generated_text},
]

In [36]:
tokenizer.apply_chat_template(
    new_msg, 
    tokenize=False, 
    #add_generation_prompt=True
)

'<｜begin▁of▁sentence｜><｜User｜>What is the most populous city in the state where the author of The Call of the Wild was born in?<｜Assistant｜>\n\nThe most populous city in the state where Jack London, the author of "The Call of the Wild," was born is Los Angeles. \n\n**Answer:** Los Angeles<｜end▁of▁sentence｜>'

In [38]:
new_msg

[{'role': 'user',
  'content': 'What is the most populous city in the state where the author of The Call of the Wild was born in?'},
 {'role': 'assistant',
  'content': '<think>Okay, so I need to figure out the most populous city in the state where the author of "The Call of the Wild" was born. Let me break this down step by step.\n\nFirst, I should identify who the author of "The Call of the Wild" is. I remember that Jack London wrote that book. Yeah, that\'s right. So, Jack London is the author.\n\nNext, I need to find out where Jack London was born. I think he was an American author, so he was born in the United States. I\'m not exactly sure of the state, but I believe it\'s California. Wait, wasn\'t he born in San Francisco? I think that\'s correct. So, his birthplace is San Francisco, California.\n\nNow, the question is asking for the most populous city in the state where he was born. Since he was born in California, I need to determine the most populous city in California. I know

In [37]:
tokenizer.apply_chat_template(
    new_msg, 
    tokenize=False, 
    add_generation_prompt=True
)

'<｜begin▁of▁sentence｜><｜User｜>What is the most populous city in the state where the author of The Call of the Wild was born in?<｜Assistant｜>\n\nThe most populous city in the state where Jack London, the author of "The Call of the Wild," was born is Los Angeles. \n\n**Answer:** Los Angeles<｜end▁of▁sentence｜><｜Assistant｜><think>\n'